In [4]:
import pyawr.mwoffice as mwo

import numpy as np
from numpy.polynomial.polynomial import polyfit

import matplotlib.pyplot as plt

#***PICK ONE***
#Agg rendering embedded in a Jupyter widget. (inline) Requires ipympl:
# %matplotlib ipympl 
#Agg rendering to a Tk canvas (new window) Requires TkInter:
%matplotlib tk 

In [5]:
awrde = mwo.CMWOffice() #Create awrde object
awrde.Project.Simulator.Analyze() #Invoke circuit simulator analysis

In [3]:
def reset_freqs (l_bnd=4e9, u_bnd=8e9, steps=10000):
    '''Reset the MWO project frequencies.

    returns an array running from 
    lower_bound to upper_bound in steps steps.'''
    
    awrde.Project.Frequencies.Clear() # clear the frequencies specified for the project
    freq_arr = np.linspace(l_bnd, u_bnd, steps)
    awrde.Project.Frequencies.AddMultiple(freq_arr) # add the frequencies in the passed frequency array

    awrde.Project.Simulator.Analyze() # must run this every time after setting or changing things in MWO project!

    return freq_arr

In [4]:
def set_circ_params(circ_name='Sample_Subcircuit', get_params=False, **kwargs): 
    '''Sets specified circuit parameters in circuit called circ_name.
    

    If you attempt to pass a parameter that doesn't exit in the circuit called circ_name, 
    MWO will throw an error.

    Returns a dictionary with the new values.
    '''

    passed_circ_param_vals = { # dictionary of subcircuit element parameter values (e.g. the value of the capacitor (element's) capacitance (parameter.))
        'CAP': awrde.Project.Schematics(circ_name).Elements(1).Parameters(2).ValueAsDouble,
        'IND': awrde.Project.Schematics(circ_name).Elements(2).Parameters(2).ValueAsDouble,
        'RES': awrde.Project.Schematics(circ_name).Elements(3).Parameters(2).ValueAsDouble
    }
    passed_circ_params = { # dictionary of subcircuit component 
        'CAP': awrde.Project.Schematics(circ_name).Elements(1).Parameters(2),
        'IND': awrde.Project.Schematics(circ_name).Elements(2).Parameters(2),
        'RES': awrde.Project.Schematics(circ_name).Elements(3).Parameters(2)
    }

    # print("DEBUG: Setting:" + kwargs.__str__())

    new_circ_param_vals = {**passed_circ_param_vals, **kwargs} # in the case of duplicate keys, only the later key-value pair is preserved
    # print("DEBUG: new parameters:" + new_circ_param_vals.__str__())

    for i, value in enumerate(new_circ_param_vals.values()):
        list(passed_circ_params.values())[i].ValueAsDouble = value # in python3, dict.keys(), .values(), and .items() return dynamically changing view objects, but not the objects themselves. Hence, list().

        # print("DEBUG: " + list(passed_circ_param_vals)[i] + " set to " + str(value))
    
    if not(get_params):
        awrde.Project.Simulator.Analyze() # must run this every time after setting or changing things in MWO project!

    return new_circ_param_vals

In [8]:
def create_polar_graph(schem_name='Hanger_Test'):
    graphs = awrde.Project.Graphs
    # TODO: Name helper graph appropriately
    graph = graphs.Add("CalcHelper {}".format(schem_name), mwo.mwGraphType.mwGT_Polar)

    graph.Measurements.Add(schem_name, 'S(2,1)')

    return graph

'CalcHelper Hanger_Test'

In [ ]:
def get_meas_vals(graph=None):
    #TODO: iterate over all measurement indicies, add a try-catch for when there's only 1 measurement on the graph
    meas = graph.Measurements[0]

    meas.XValues

In [ ]:
gr = create_polar_graph()
get_meas_vals(gr)